# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 9 2022 12:05PM,243773,20,8053067,"ACI Healthcare USA, Inc.",Released
1,Aug 9 2022 12:05PM,243773,20,8070453,"ACI Healthcare USA, Inc.",Released
2,Aug 9 2022 12:05PM,243773,20,8070465,"ACI Healthcare USA, Inc.",Released
3,Aug 9 2022 12:05PM,243773,20,8070484,"ACI Healthcare USA, Inc.",Released
4,Aug 9 2022 12:05PM,243773,20,8070473-BO,"ACI Healthcare USA, Inc.",Released
5,Aug 9 2022 12:05PM,243773,20,8070480-BO,"ACI Healthcare USA, Inc.",Released
6,Aug 9 2022 11:56AM,243772,16,8377462,Sartorius Lab Products and Service,Released
7,Aug 9 2022 11:56AM,243772,16,8377484,Sartorius Lab Products and Service,Released
8,Aug 9 2022 11:56AM,243772,16,8380604,Sartorius Bioprocess Solutions,Released
9,Aug 9 2022 11:56AM,243771,10,0085930518,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,243768,Released,19
28,243770,Released,1
29,243771,Released,6
30,243772,Released,3
31,243773,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243768,NaN,6.0,19.0
243770,NaN,NaN,1.0
243771,NaN,NaN,6.0
243772,NaN,NaN,3.0
243773,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243682,0.0,0.0,1.0
243687,13.0,1.0,1.0
243721,0.0,0.0,1.0
243722,0.0,0.0,1.0
243723,0.0,32.0,76.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243682,0,0,1
243687,13,1,1
243721,0,0,1
243722,0,0,1
243723,0,32,76


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243682,0,0,1
1,243687,13,1,1
2,243721,0,0,1
3,243722,0,0,1
4,243723,0,32,76


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243682,,,1
1,243687,13,1,1
2,243721,,,1
3,243722,,,1
4,243723,,32,76


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 9 2022 12:05PM,243773,20,"ACI Healthcare USA, Inc."
6,Aug 9 2022 11:56AM,243772,16,Sartorius Lab Products and Service
8,Aug 9 2022 11:56AM,243772,16,Sartorius Bioprocess Solutions
9,Aug 9 2022 11:56AM,243771,10,ISDIN Corporation
15,Aug 9 2022 11:48AM,243770,20,"HVL, LLC dba Atrium Innovations"
16,Aug 9 2022 11:32AM,243768,19,"AdvaGen Pharma, Ltd"
41,Aug 9 2022 11:00AM,243767,10,MedStone Pharma LLC
50,Aug 9 2022 10:47AM,243764,10,Beach Patient Assistance
64,Aug 9 2022 10:46AM,243765,19,"NAPP Technologies, LLC"
65,Aug 9 2022 10:09AM,243757,15,"Mizner Bioscience, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 9 2022 12:05PM,243773,20,"ACI Healthcare USA, Inc.",,,6
1,Aug 9 2022 11:56AM,243772,16,Sartorius Lab Products and Service,,,3
2,Aug 9 2022 11:56AM,243772,16,Sartorius Bioprocess Solutions,,,3
3,Aug 9 2022 11:56AM,243771,10,ISDIN Corporation,,,6
4,Aug 9 2022 11:48AM,243770,20,"HVL, LLC dba Atrium Innovations",,,1
5,Aug 9 2022 11:32AM,243768,19,"AdvaGen Pharma, Ltd",,6,19
6,Aug 9 2022 11:00AM,243767,10,MedStone Pharma LLC,,,9
7,Aug 9 2022 10:47AM,243764,10,Beach Patient Assistance,,,14
8,Aug 9 2022 10:46AM,243765,19,"NAPP Technologies, LLC",,,1
9,Aug 9 2022 10:09AM,243757,15,"Mizner Bioscience, LLC",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 12:05PM,243773,20,"ACI Healthcare USA, Inc.",6,,
1,Aug 9 2022 11:56AM,243772,16,Sartorius Lab Products and Service,3,,
2,Aug 9 2022 11:56AM,243772,16,Sartorius Bioprocess Solutions,3,,
3,Aug 9 2022 11:56AM,243771,10,ISDIN Corporation,6,,
4,Aug 9 2022 11:48AM,243770,20,"HVL, LLC dba Atrium Innovations",1,,
5,Aug 9 2022 11:32AM,243768,19,"AdvaGen Pharma, Ltd",19,6,
6,Aug 9 2022 11:00AM,243767,10,MedStone Pharma LLC,9,,
7,Aug 9 2022 10:47AM,243764,10,Beach Patient Assistance,14,,
8,Aug 9 2022 10:46AM,243765,19,"NAPP Technologies, LLC",1,,
9,Aug 9 2022 10:09AM,243757,15,"Mizner Bioscience, LLC",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 12:05PM,243773,20,"ACI Healthcare USA, Inc.",6,,
1,Aug 9 2022 11:56AM,243772,16,Sartorius Lab Products and Service,3,,
2,Aug 9 2022 11:56AM,243772,16,Sartorius Bioprocess Solutions,3,,
3,Aug 9 2022 11:56AM,243771,10,ISDIN Corporation,6,,
4,Aug 9 2022 11:48AM,243770,20,"HVL, LLC dba Atrium Innovations",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 12:05PM,243773,20,"ACI Healthcare USA, Inc.",6.0,NaN,NaN
1,Aug 9 2022 11:56AM,243772,16,Sartorius Lab Products and Service,3.0,NaN,NaN
2,Aug 9 2022 11:56AM,243772,16,Sartorius Bioprocess Solutions,3.0,NaN,NaN
3,Aug 9 2022 11:56AM,243771,10,ISDIN Corporation,6.0,NaN,NaN
4,Aug 9 2022 11:48AM,243770,20,"HVL, LLC dba Atrium Innovations",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 12:05PM,243773,20,"ACI Healthcare USA, Inc.",6.0,0.0,0.0
1,Aug 9 2022 11:56AM,243772,16,Sartorius Lab Products and Service,3.0,0.0,0.0
2,Aug 9 2022 11:56AM,243772,16,Sartorius Bioprocess Solutions,3.0,0.0,0.0
3,Aug 9 2022 11:56AM,243771,10,ISDIN Corporation,6.0,0.0,0.0
4,Aug 9 2022 11:48AM,243770,20,"HVL, LLC dba Atrium Innovations",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2437412,125.0,1.0,0.0
15,974959,107.0,50.0,0.0
16,487544,6.0,0.0,0.0
19,975022,21.0,7.0,0.0
20,974974,14.0,10.0,13.0
21,487443,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2437412,125.0,1.0,0.0
1,15,974959,107.0,50.0,0.0
2,16,487544,6.0,0.0,0.0
3,19,975022,21.0,7.0,0.0
4,20,974974,14.0,10.0,13.0
5,21,487443,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,125.0,1.0,0.0
1,15,107.0,50.0,0.0
2,16,6.0,0.0,0.0
3,19,21.0,7.0,0.0
4,20,14.0,10.0,13.0
5,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,125.0
1,15,Released,107.0
2,16,Released,6.0
3,19,Released,21.0
4,20,Released,14.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,0.0,13.0,0.0
Executing,1.0,50.0,0.0,7.0,10.0,0.0
Released,125.0,107.0,6.0,21.0,14.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,13.0,0.0
1,Executing,1.0,50.0,0.0,7.0,10.0,0.0
2,Released,125.0,107.0,6.0,21.0,14.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,13.0,0.0
1,Executing,1.0,50.0,0.0,7.0,10.0,0.0
2,Released,125.0,107.0,6.0,21.0,14.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()